In [2]:
import pandas as pd
import numpy as np 
from Bio.PDB import *

In [3]:
phosphosite_data = pd.read_csv("~/pnnl/ppi/phospho_site_plus_human_20240422.tsv", delimiter='\t')
pka_data = pd.read_csv("~/pnnl/ppi/all_pka.csv", header=0)

In [4]:
pka_data = pka_data.drop(columns=["Unnamed: 0"])
pka_data

,uniprotID,AA,res_number,pK,state,position
0,A0PJZ3,NTR,5001,6.891614,undefined,5001.0
1,A0PJZ3,LYS,2,10.328119,protonated,2.0
2,A0PJZ3,SER,5,NaN,undefined,5.0
3,A0PJZ3,LYS,6,10.375520,protonated,6.0
4,A0PJZ3,SER,24,NaN,undefined,24.0
...,...,...,...,...,...,...
5916609,P51587,ASP,1397,3.790052,deprotonated,NaN
5916610,P51587,CYS,1398,9.230363,protonated,NaN
5916611,P51587,THR,1399,NaN,undefined,NaN
5916612,P51587,CTR,6400,2.997512,deprotonated,NaN


In [5]:
phosphosite_data


,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,LT_LIT,MS_LIT,MS_CST,CST_CAT#,Ambiguous_Site
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,NaN,3.0,1.0,NaN,0
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,NaN,8.0,NaN,NaN,0
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,NaN,NaN,4.0,NaN,0
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,NaN,NaN,1.0,NaN,0
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,NaN,4.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389029,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,NaN,1.0,NaN,NaN,0
389030,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,NaN,1.0,NaN,NaN,0
389031,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,NaN,1.0,NaN,NaN,0
389032,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,1.0,NaN,NaN,NaN,0


In [6]:
# clean phosphosite_data
phosphosite_data['RES_NUM'] = phosphosite_data['MOD_RSD'].str.split('-').str[0].str[1:] # split string on '-' and get rid of first letter
phosphosite_data['PKA_ID'] = phosphosite_data['ACC_ID'] +"_"+ phosphosite_data['RES_NUM']
phosphosite_data


,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,LT_LIT,MS_LIT,MS_CST,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,NaN,3.0,1.0,NaN,0,2,P31946_2
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,NaN,8.0,NaN,NaN,0,6,P31946_6
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,NaN,NaN,4.0,NaN,0,21,P31946_21
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,NaN,NaN,1.0,NaN,0,32,P31946_32
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,NaN,4.0,NaN,NaN,0,39,P31946_39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389029,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,NaN,1.0,NaN,NaN,0,450,Q9UKY1_450
389030,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,NaN,1.0,NaN,NaN,0,891,Q9Y2X9_891
389031,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,NaN,1.0,NaN,NaN,0,1196,O15014_1196
389032,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,1.0,NaN,NaN,NaN,0,169,Q15942_169


In [7]:
pka_data['PKA_ID'] = pka_data['uniprotID'].str.split('-').str[0] +"_"+ pka_data['res_number'].apply(str)
pka_data

,uniprotID,AA,res_number,pK,state,position,PKA_ID
0,A0PJZ3,NTR,5001,6.891614,undefined,5001.0,A0PJZ3_5001
1,A0PJZ3,LYS,2,10.328119,protonated,2.0,A0PJZ3_2
2,A0PJZ3,SER,5,NaN,undefined,5.0,A0PJZ3_5
3,A0PJZ3,LYS,6,10.375520,protonated,6.0,A0PJZ3_6
4,A0PJZ3,SER,24,NaN,undefined,24.0,A0PJZ3_24
...,...,...,...,...,...,...,...
5916609,P51587,ASP,1397,3.790052,deprotonated,NaN,P51587_1397
5916610,P51587,CYS,1398,9.230363,protonated,NaN,P51587_1398
5916611,P51587,THR,1399,NaN,undefined,NaN,P51587_1399
5916612,P51587,CTR,6400,2.997512,deprotonated,NaN,P51587_6400


In [8]:
len(pka_data['PKA_ID'].unique())

4350682

In [9]:
len(pka_data['uniprotID'].unique())

20595

In [10]:
full_data = pd.merge(phosphosite_data, pka_data, on="PKA_ID", how = "left")
print(full_data.shape)
full_data

(436145, 23)


,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,...,NaN,0,2,P31946_2,P31946,THR,2.0,NaN,undefined,2.0
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,...,NaN,0,6,P31946_6,P31946,SER,6.0,NaN,undefined,6.0
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,...,NaN,0,21,P31946_21,P31946,TYR,21.0,11.100927,protonated,21.0
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,...,NaN,0,32,P31946_32,P31946,THR,32.0,NaN,undefined,32.0
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,...,NaN,0,39,P31946_39,P31946,SER,39.0,NaN,undefined,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436140,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,...,NaN,0,450,Q9UKY1_450,Q9UKY1,SER,450.0,NaN,undefined,450.0
436141,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,...,NaN,0,891,Q9Y2X9_891,Q9Y2X9,SER,891.0,NaN,undefined,891.0
436142,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,...,NaN,0,1196,O15014_1196,O15014,SER,1196.0,NaN,undefined,1196.0
436143,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,...,NaN,0,169,Q15942_169,Q15942,SER,169.0,NaN,undefined,169.0


In [11]:
full_noduplicates_data = full_data.drop_duplicates()
full_noduplicates_data

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,...,NaN,0,2,P31946_2,P31946,THR,2.0,NaN,undefined,2.0
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,...,NaN,0,6,P31946_6,P31946,SER,6.0,NaN,undefined,6.0
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,...,NaN,0,21,P31946_21,P31946,TYR,21.0,11.100927,protonated,21.0
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,...,NaN,0,32,P31946_32,P31946,THR,32.0,NaN,undefined,32.0
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,...,NaN,0,39,P31946_39,P31946,SER,39.0,NaN,undefined,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436140,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,...,NaN,0,450,Q9UKY1_450,Q9UKY1,SER,450.0,NaN,undefined,450.0
436141,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,...,NaN,0,891,Q9Y2X9_891,Q9Y2X9,SER,891.0,NaN,undefined,891.0
436142,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,...,NaN,0,1196,O15014_1196,O15014,SER,1196.0,NaN,undefined,1196.0
436143,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,...,NaN,0,169,Q15942_169,Q15942,SER,169.0,NaN,undefined,169.0


In [12]:
print(pka_data.shape)
print(phosphosite_data.shape)
print(full_noduplicates_data.shape) # 1468 of them didn't match up 
# I guess this makes sense. pka didn't output for ALL residues, and that's not that large of a fraction


(5916614, 7)
(389034, 17)
(436012, 23)


In [13]:
full_noduplicates_data['PKA_ID'].nunique()

370369

In [14]:
full_noduplicates_data['PKA_ID'].value_counts()

PKA_ID
Q8WZ42_1054    91
Q8WZ42_565     89
Q8WZ42_308     87
Q8WZ42_1159    86
Q8WZ42_300     86
               ..
O75592_2359     1
O75592_2353     1
O75592_2348     1
O75592_2344     1
Q15942_246      1
Name: count, Length: 370369, dtype: int64

In [15]:
print(full_data['pK'].notna().sum(), "rows have pka")
print(189514/425877,"have pka annotations")

189765 rows have pka
0.44499702965879817 have pka annotations


In [16]:
# adding the interfaces data
interfaces_data = pd.read_csv("~/pnnl/ppi/2024.05.28_interface_summary_5A.tsv", delimiter='\t', header=0)
interfaces_data

/var/folders/18/kpy7dqgx6kd36fdl425cpnkr0000gn/T/ipykernel_50212/3260714051.py:2: DtypeWarning: Columns (2,3,4,5,6,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interfaces_data = pd.read_csv("~/pnnl/ppi/2024.05.28_interface_summary_5A.tsv", delimiter='\t', header=0)


,interaction_id,pdockq,uniprot_id1,uniprot_id2,chain1,chain2,ifresid1,ifresid2,sources,n_references,pdb
0,O75106_Q16853,0.74,O75106,Q16853,A,B,"R169,A203,A204,V205,H206,L212,R213,W220,N226,I...","P39,V209,L218,Q219,W226,N232,I233,S234,G235,A2...","BioGRID,humap,intact,string",2,O75106/O75106_Q16853.pdb
1,Q15118_Q15118,0.73,Q15118,Q15118,A,B,"S53,P54,P56,Y179,D182,R183,M186,L255,A257,H304...","S53,P54,P56,Y179,D182,R183,M186,E253,L255,A257...","BioGRID,intact",2,Q15118/Q15118_Q15118.pdb
2,P11142_Q92598,0.73,P11142,Q92598,A,B,"K25,E27,I28,A30,N31,D32,Q33,G34,R36,E48,L50,D5...","R19,A27,N28,E29,F30,S31,R33,N54,T58,Y184,R261,...","BioGRID,corum,humap,intact,otar,string,xlinkdb",9,P11142/P11142_Q92598.pdb
3,Q13326_Q16585,0.73,Q13326,Q16585,A,B,"V40,L41,L43,L44,L47,V48,N50,L51,T54,I55,L58,F6...","V68,I69,L71,L72,L75,A76,I78,N79,I82,I86,M100,F...","corum,otar,string",0,Q13326/Q13326_Q16585.pdb
4,Q13326_Q92629,0.73,Q13326,Q92629,A,B,"K33,L36,Y37,V40,L41,L43,L44,L47,V48,N50,L51,T5...","R30,K31,C33,L34,F37,V38,L40,L41,L44,I45,V47,N4...","corum,string",0,Q13326/Q13326_Q92629.pdb
...,...,...,...,...,...,...,...,...,...,...,...
486094,P23193_Q92889,0.00,NaN,NaN,NaN,NaN,NaN,NaN,otar,0,NaN
486095,P23193_Q92541,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"BioGRID,intact,otar,string",1,NaN
486096,P23193_Q8WX92,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"otar,string",0,NaN
486097,P23193_Q8WVC0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"BioGRID,intact,otar,string",2,NaN


In [17]:
# adding pockets data
pockets_data = pd.read_csv("~/pnnl/ppi/2024.05.28_pockets.tsv", delimiter='\t', header=0)
pockets_data

,struct_id,pocket_id,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_resid,pocket_plddt_mean,pocket_score_combined_scaled
0,A0A024R1R8-F1,1,4.042788,0.316535,0.772959,"{21,22,23,24,25,26,28,29,32}",83.937778,283.034096
1,A0A024R1R8-F1,2,3.175737,0.347111,0.808219,"{12,13,14,15,16,17}",61.206667,102.718057
2,A0A024RBG1-F1,1,7.310256,0.435597,0.856184,"{2,3,4,5,6,7,8,9,10,18,20,21,22,39,40,41,42,47...",89.456190,979.457587
3,A0A024RBG1-F1,2,6.350910,0.389675,0.814896,"{54,57,58,60,61,62,64,65,67,68,69,73,74,75,76,...",83.186923,938.222063
4,A0A024RBG1-F1,3,3.827945,0.378204,0.806045,"{1,2,3,4,5,6,109,110,112,113,114}",77.053636,422.703190
...,...,...,...,...,...,...,...,...
547396,X6R8D5-F1,3,3.894257,0.338401,0.777778,"{86,87,88,89,90,91,92,93,98,100,101,102,103}",56.513846,99.047598
547397,X6R8D5-F1,4,4.196873,0.328247,0.768473,"{43,44,45,46,47,48,49,50,127}",59.902222,107.547205
547398,X6R8D5-F1,5,4.465454,0.314353,0.751790,"{81,87,90,91,92,93,99,100,101,102,103,104,105}",58.298462,95.185313
547399,X6R8D5-F1,6,3.198691,0.398147,0.825342,"{66,67,68,69,71,72,73,75,76,77}",61.416000,122.028350


In [18]:
pockets_data["struct_id"].str.split('-')

0         [A0A024R1R8, F1]
1         [A0A024R1R8, F1]
2         [A0A024RBG1, F1]
3         [A0A024RBG1, F1]
4         [A0A024RBG1, F1]
                ...       
547396        [X6R8D5, F1]
547397        [X6R8D5, F1]
547398        [X6R8D5, F1]
547399        [X6R8D5, F1]
547400        [X6R8D5, F1]
Name: struct_id, Length: 547401, dtype: object

In [19]:
sum(full_noduplicates_data["ACC_ID"].str.contains("F1", case=False, na=False).astype(int))
# we don't have F1 at the end of some of the uniprot-IDs

674

In [20]:
sum(pockets_data["struct_id"].str.contains("F1", case=False, na=False).astype(int))

547401

In [21]:
# getting rid of F1 in the data
pockets_data['uniprot_id'] = pockets_data['struct_id'].str.split('-').str[0]
full_noduplicates_data['uniprot_id'] = full_noduplicates_data['ACC_ID'].str.split('-').str[0]
full_noduplicates_data.drop(columns = ['uniprotID'])

/var/folders/18/kpy7dqgx6kd36fdl425cpnkr0000gn/T/ipykernel_50212/37065774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_noduplicates_data['uniprot_id'] = full_noduplicates_data['ACC_ID'].str.split('-').str[0]


,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,CST_CAT#,Ambiguous_Site,RES_NUM,PKA_ID,AA,res_number,pK,state,position,uniprot_id
0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,______MtMDksELV,...,NaN,0,2,P31946_2,THR,2.0,NaN,undefined,2.0,P31946
1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,__MtMDksELVQkAk,...,NaN,0,6,P31946_6,SER,6.0,NaN,undefined,6.0,P31946
2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,LAEQAERyDDMAAAM,...,NaN,0,21,P31946_21,TYR,21.0,11.100927,protonated,21.0,P31946
3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,AAAMkAVtEQGHELs,...,NaN,0,32,P31946_32,THR,32.0,NaN,undefined,32.0,P31946
4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,tEQGHELsNEERNLL,...,NaN,0,39,P31946_39,SER,39.0,NaN,undefined,39.0,P31946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436140,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,ATAAVPTsQSVkHET,...,NaN,0,450,Q9UKY1_450,SER,450.0,NaN,undefined,450.0,Q9UKY1
436141,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,TRVktPTsQSYR___,...,NaN,0,891,Q9Y2X9_891,SER,891.0,NaN,undefined,891.0,Q9Y2X9
436142,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,SDCKLPTsEESRLGS,...,NaN,0,1196,O15014_1196,SER,1196.0,NaN,undefined,1196.0,O15014
436143,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,DPFkARVssGyVPPP,...,NaN,0,169,Q15942_169,SER,169.0,NaN,undefined,169.0,Q15942


In [22]:
pockets_data

,struct_id,pocket_id,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_resid,pocket_plddt_mean,pocket_score_combined_scaled,uniprot_id
0,A0A024R1R8-F1,1,4.042788,0.316535,0.772959,"{21,22,23,24,25,26,28,29,32}",83.937778,283.034096,A0A024R1R8
1,A0A024R1R8-F1,2,3.175737,0.347111,0.808219,"{12,13,14,15,16,17}",61.206667,102.718057,A0A024R1R8
2,A0A024RBG1-F1,1,7.310256,0.435597,0.856184,"{2,3,4,5,6,7,8,9,10,18,20,21,22,39,40,41,42,47...",89.456190,979.457587,A0A024RBG1
3,A0A024RBG1-F1,2,6.350910,0.389675,0.814896,"{54,57,58,60,61,62,64,65,67,68,69,73,74,75,76,...",83.186923,938.222063,A0A024RBG1
4,A0A024RBG1-F1,3,3.827945,0.378204,0.806045,"{1,2,3,4,5,6,109,110,112,113,114}",77.053636,422.703190,A0A024RBG1
...,...,...,...,...,...,...,...,...,...
547396,X6R8D5-F1,3,3.894257,0.338401,0.777778,"{86,87,88,89,90,91,92,93,98,100,101,102,103}",56.513846,99.047598,X6R8D5
547397,X6R8D5-F1,4,4.196873,0.328247,0.768473,"{43,44,45,46,47,48,49,50,127}",59.902222,107.547205,X6R8D5
547398,X6R8D5-F1,5,4.465454,0.314353,0.751790,"{81,87,90,91,92,93,99,100,101,102,103,104,105}",58.298462,95.185313,X6R8D5
547399,X6R8D5-F1,6,3.198691,0.398147,0.825342,"{66,67,68,69,71,72,73,75,76,77}",61.416000,122.028350,X6R8D5


# The procedure:

PSP, pocket, interface

start  with PSP:

loop through unique uniprot id

for each unique uniprot id

select all the PTM sites in the same protein from PSP
select all pockets ..... from procket table
select all interfaces .... from interface table (any pair has uniprot id)

loop through ptm  sites

for each site, 

calculate prockets distance
if site in any pockets: (for loop)
    mindistance = 0
    in pockets = True
    pockets number = [#]
else 
    in pcikates = False
loop through all the pockets
calculate the first taken as minial
       minidistance = #
       pockets number = #
 calcualte any pairt betwene site and pocket residue
 compare the new distance to the minidistant 
  if < 
    replace
    elif =
    append
    
    




- match up uniprot id from pockets data to ptm 
- check if any ptms are located inside of the pocket 
- new column w/ if inside the pocket T/F
-if FALSE, find the closest PTM 


-new column w/ distance from ptm and closest pocket residue

### Pockets Data

In [23]:
# making a smaller test dataset
test_psp = full_noduplicates_data.sample(n = 300)
test_psp['closest_pocket'] = "NaN"
test_psp['inside_pocket'] = "NaN"
test_psp['distance_from_pocket'] = "NaN"
test_psp

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_pocket,inside_pocket,distance_from_pocket
181867,PRPF8,PRPF8,Q6P2Q9,17p13.3,S1358-p,4772824,human,273.60,NaN,THFRsGMsHEEDQLI,...,Q6P2Q9,SER,1358.0,NaN,undefined,1358.0,Q6P2Q9,NaN,NaN,NaN
103795,HSF1,HSF1,Q00613,8q24.3,S216-p,4668102,human,57.26,NaN,IPLMLNDsGsAHSMP,...,Q00613,SER,216.0,NaN,undefined,216.0,Q00613,NaN,NaN,NaN
46170,CHD6,CHD-6,Q8TD26,20q12,S85-p,36123599,human,305.41,NaN,sHNGMEDsGGGGTGV,...,Q8TD26,ASP,85.0,2.556353,deprotonated,685.0,Q8TD26,NaN,NaN,NaN
142689,MUC16,MUC16,Q8WXI7,19p13.2,T1002-p,25308906,human,1519.17,NaN,TVMVSKFtSPATsSM,...,Q8WXI7-F5,THR,1002.0,NaN,undefined,11602.0,Q8WXI7,NaN,NaN,NaN
5782,AFAP1,AFAP,Q8N556,4p16.1,T524-p,36420015,human,80.72,NaN,PVAsNGVtGKGkTLS,...,Q8N556,THR,524.0,NaN,undefined,524.0,Q8N556,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292009,RSBN1L,RSBN1L,Q6PCB5,7q11.23,K191-ac,41519666,human,94.87,NaN,REENGEVkPLPRDKI,...,Q6PCB5,LYS,191.0,10.710749,protonated,191.0,Q6PCB5,NaN,NaN,NaN
236746,TTN,Titin,Q8WZ42,2q31.2,T811-p,4693555,human,3816.03,NaN,HVDkRPRtAsPHFtV,...,Q8WZ42-F2,GLU,811.0,2.976560,deprotonated,5411.0,Q8WZ42,NaN,NaN,NaN
78908,RMDN3,FAM82A2,Q96TC7,15q15.1,S193-p,4720649,human,52.12,NaN,ERDsDKEsEDGEDEV,...,Q96TC7,SER,193.0,NaN,undefined,193.0,Q96TC7,NaN,NaN,NaN
243533,TPM1,TPM1,P09493,15q22.2,S229-p,36839451,human,32.71,Tropomyosin,EEEIkVLsDkLkEAE,...,P09493,SER,229.0,NaN,undefined,229.0,P09493,NaN,NaN,NaN


In [24]:
unique_uniprots = test_psp['uniprot_id'].unique() # get all of the unique uniprot

# for each unique uniprotID...
for uniprot in unique_uniprots:
    # isolate to psp and pockets in each uniprot
    psp_only_uniprot = test_psp[test_psp.uniprot_id == uniprot]
    pocket_only_uniprot = pockets_data[pockets_data.uniprot_id == uniprot]
    # print(pocket_only_uniprot)

# for each psp
    for phosphosite_row_index in psp_only_uniprot.index:
        #print(psp_only_uniprot)
        #print(phosphosite_row_index)
        residue_num = str(psp_only_uniprot.loc[phosphosite_row_index,'res_number']) # finding the residue number of the psp
        #print(residue_num)
        # use the residue # to get the coordinates in space from pdb file
        
        for pocket_index in pocket_only_uniprot.index : # get all the residues in all of the pockets 
            pocket_residues = pocket_only_uniprot.loc[pocket_index,'pocket_resid']
            # print(pocket_residues)

            # check if it's inside of a pocket
            identifier = uniprot + "_" + str(residue_num)
            if residue_num in pocket_residues:
                test_psp.loc[phosphosite_row_index,'inside_pocket'] = 1 # if residue is in the pocket, put a 1 in the inside pocket column
            else:
                test_psp.loc[phosphosite_row_index,'inside_pocket'] = 0
                


test_psp           

,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,SITE_+/-7_AA,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_pocket,inside_pocket,distance_from_pocket
181867,PRPF8,PRPF8,Q6P2Q9,17p13.3,S1358-p,4772824,human,273.60,NaN,THFRsGMsHEEDQLI,...,Q6P2Q9,SER,1358.0,NaN,undefined,1358.0,Q6P2Q9,NaN,0,NaN
103795,HSF1,HSF1,Q00613,8q24.3,S216-p,4668102,human,57.26,NaN,IPLMLNDsGsAHSMP,...,Q00613,SER,216.0,NaN,undefined,216.0,Q00613,NaN,0,NaN
46170,CHD6,CHD-6,Q8TD26,20q12,S85-p,36123599,human,305.41,NaN,sHNGMEDsGGGGTGV,...,Q8TD26,ASP,85.0,2.556353,deprotonated,685.0,Q8TD26,NaN,NaN,NaN
142689,MUC16,MUC16,Q8WXI7,19p13.2,T1002-p,25308906,human,1519.17,NaN,TVMVSKFtSPATsSM,...,Q8WXI7-F5,THR,1002.0,NaN,undefined,11602.0,Q8WXI7,NaN,NaN,NaN
5782,AFAP1,AFAP,Q8N556,4p16.1,T524-p,36420015,human,80.72,NaN,PVAsNGVtGKGkTLS,...,Q8N556,THR,524.0,NaN,undefined,524.0,Q8N556,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292009,RSBN1L,RSBN1L,Q6PCB5,7q11.23,K191-ac,41519666,human,94.87,NaN,REENGEVkPLPRDKI,...,Q6PCB5,LYS,191.0,10.710749,protonated,191.0,Q6PCB5,NaN,0,NaN
236746,TTN,Titin,Q8WZ42,2q31.2,T811-p,4693555,human,3816.03,NaN,HVDkRPRtAsPHFtV,...,Q8WZ42-F2,GLU,811.0,2.976560,deprotonated,5411.0,Q8WZ42,NaN,NaN,NaN
78908,RMDN3,FAM82A2,Q96TC7,15q15.1,S193-p,4720649,human,52.12,NaN,ERDsDKEsEDGEDEV,...,Q96TC7,SER,193.0,NaN,undefined,193.0,Q96TC7,NaN,0,NaN
243533,TPM1,TPM1,P09493,15q22.2,S229-p,36839451,human,32.71,Tropomyosin,EEEIkVLsDkLkEAE,...,P09493,SER,229.0,NaN,undefined,229.0,P09493,NaN,NaN,NaN


,struct_id,pocket_id,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_resid,pocket_plddt_mean,pocket_score_combined_scaled,uniprot_id
523800,Q9UPQ9-F1,1,4.927090,0.353983,0.761453,"{1268,1269,1270,1271,1272,1273,1274,1275,1276,...",50.952500,231.993243,Q9UPQ9
523801,Q9UPQ9-F1,2,6.377510,0.334456,0.740651,"{780,781,782,783,784,785,786,1001,1002,1003,10...",35.053500,78.204325,Q9UPQ9
523802,Q9UPQ9-F1,3,4.307902,0.376128,0.782696,"{908,909,910,911,912,913,914,934,935,936,957,9...",35.767333,70.352341,Q9UPQ9
523803,Q9UPQ9-F1,4,6.160578,0.326519,0.745763,"{910,911,912,913,914,915,916,917,932,933,934,9...",34.287143,51.947639,Q9UPQ9
523804,Q9UPQ9-F1,5,4.344631,0.331235,0.786427,"{966,967,968,1245,1246,1247,1248,1249,1250,1251}",33.555000,49.391743,Q9UPQ9
523805,Q9UPQ9-F1,6,5.805484,0.320993,0.765507,"{754,755,756,757,758,759,760,761,762,763,764,7...",32.410714,41.425192,Q9UPQ9
523806,Q9UPQ9-F1,7,5.093673,0.364072,0.789007,"{782,783,784,785,997,998,999,1000,1001,1002,10...",31.950000,42.576901,Q9UPQ9
523807,Q9UPQ9-F1,8,4.099537,0.361961,0.773756,"{1476,1478,1479,1480,1481,1482,1483,1484,1485,...",57.294545,157.739524,Q9UPQ9
523808,Q9UPQ9-F1,9,4.259166,0.344195,0.769565,"{798,799,800,801,802,803,804,811,812,813,814,8...",33.338462,41.789425,Q9UPQ9
523809,Q9UPQ9-F1,10,4.328579,0.313602,0.716738,"{1659,1660,1661,1662,1665,1677,1678,1679,1681,...",82.500000,506.611768,Q9UPQ9
